<a href="https://colab.research.google.com/github/abdullahyahyamohammed/AI_misogyny/blob/main/misogyny.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# BUILDING NLP WEB APPS WITH GRADIO AND HUGGING FACE TRANSFORMERS

Web app design/deployment is arguably one of the biggest weaknesses among data scientists and analysts. But there's no avoiding it if you need to share an NLP/ML demo with colleagues or a client.

Even a simple web app can go a long way towards demonstrating how your solution works, and what the results look like. More importantly, it allows non-technical users to test the proposed solution for themselves, and see if the product is on the right track.

But a "simple" app can take hours to figure out if you are unfamiliar with HTML and the finicky steps involved in deploying to hosting services like Heroku or PythonAnywhere. It is doubly frustrating if you are merely in the early stages of development, when all you need is a frills-free demo.

Several tools and libraries have emerged in recent years to address this need, but the most user-friendly I've used to date is [Gradio](https://gradio.app/). Its close integration with [Hugging Face](https://huggingface.co/) makes it even easier to use, if you are working on a transformers-based NLP solution.

With a few lines of code, you can get a simple web app for sentiment analysis or translation up and running. You can even get away with just two lines of code, if you rely on [Hugging Face's hosted Inference API](https://huggingface.co/blog/gradio). But do note that the publicly accessible Inference API can be slow. 

What's even more interesting about the latest version of Gradio is that you can load multiple models in one web app, either in parallel or in series, ie, compare different translation models in one app, or combine translation and summary models in one app.

Over the next few notebooks, I'll share some examples of how Gradio can be used for these standalone and "chain-linked" NLP apps. They'll predominantly be transformer-based examples, thought I'll have one for scikit-learn as well, using a Logistic Regression model.

Let's start with a simple sentiment analysis web app using Hugging Face's pipeline.

In [ ]:
# pip install gradio

In [ ]:
pip install transformers

     |████████████████████████████████| 2.8 MB 14.2 MB/s 
     |████████████████████████████████| 636 kB 61.8 MB/s 
     |████████████████████████████████| 3.3 MB 84.9 MB/s 
     |████████████████████████████████| 52 kB 1.2 MB/s 
     |████████████████████████████████| 895 kB 75.8 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
import gradio as gr
import re

from transformers import pipeline

## 1. DEFINE SENTIMENT ANALYSIS FUNCTION

I'm using the Hugging Face pipeline to simplify the example. You can use your own finetuned transformer model if you wish.

The pipeline also allows you to swap out functionalities very quickly. So if I want to use the pretrained models available in the pipeline for summarization or translation, I can just easily change a few words and I would be good to go. 

See [Hugging Face's documentation](https://huggingface.co/transformers/main_classes/pipelines.html) for more details on the various NLP tasks you can execute via the pipelines.

In [ ]:
sentiment = pipeline("sentiment-analysis") 
# you can swop out "sentiment-analysis" for other task identifiers such as "summarization" or "zero-shot-classification".

# I've added optional lines for text cleaning
# note that the sentiment-analysis pipeline returns 2 values - a label and a score
def sentiment_analysis(text):
    text = text.encode("ascii", errors="ignore").decode(
        "ascii"
    )  # remove non-ascii, Chinese characters
    text = text.lower()  # lower case
    text = re.sub(r"\n", " ", text)
    text = re.sub(r"\n\n", " ", text)
    text = re.sub(r"\t", " ", text)
    text = text.strip(" ")
    text = re.sub(r"[^\w\s]", "", text)  # remove punctuation and special characters
    text = re.sub(
        " +", " ", text
    ).strip()  # get rid of multiple spaces and replace with a single
    results = sentiment(text)
    return results[0]["label"], round(results[0]["score"], 5)


No model was supplied, defaulted to distilbert-base-uncased-finetuned-sst-2-english (https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english)


Downloading:   0%|          | 0.00/629 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/268M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

## 2. DEFINE GRADIO INTERFACE

The parameters for the Gradio interface are pretty intuitive and easy to use once you've tried out an example or two on your own. See the [documentation](https://gradio.app/docs) for further details.

In [ ]:
gradio_ui = gr.Interface(
    fn=sentiment_analysis,
    title="Sentiment Analysis",
    description="Enter some text and see if the Distilbert model can gauge the sentiment correctly",
    inputs=gr.inputs.Textbox(lines=10, label="Paste some text here"),
    outputs=[
        gr.outputs.Textbox(label="Sentiment Label"),
        gr.outputs.Textbox(label="Sentiment Score"),
    ],
)


## 2.1 VOILA! 

If you've struggled to cobble together a simple web app, like I have, this is nothing short of a lifesaver.

In [ ]:
# set gradio_ui.launch(share=True) if you need to share it outside of your local machine.
# The link works for 24 hours and as long as your notebook is running

gradio_ui.launch()

Colab notebook detected. To show errors in colab notebook, set `debug=True` in `launch()`
This share link will expire in 24 hours. If you need a permanent link, visit: https://gradio.app/introducing-hosted (NEW!)
Running on External URL: https://37746.gradio.app
Interface loading below...


(<Flask 'gradio.networking'>,
 'http://127.0.0.1:7860/',
 'https://37746.gradio.app')